In [1]:
import torch
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler

import string
import time
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

%load_ext autoreload
%autoreload 2

from rnn_will import RNN, PrepareData

torch.cuda.is_available()

True

In [2]:
#input_dim = 7
"""
part d change dropout
"""
drop_ratio = 0.3
hidden_dim = 50
num_layers = 1
seq_size = 25
all_letters = string.printable
n_letters = len(all_letters)

In [3]:
dataset = PrepareData('../input.txt', seq_size, val_ratio=0.2)
#dataset = PrepareData('sample-music2.txt', seq_size)
train_data, val_data = dataset.dataset()
print(train_data.__len__(), val_data.__len__())

15688 3921


In [4]:
def inputTensor(line):
    if len(line) == 1:
        tensor = torch.zeros(1, n_letters)
        tensor[0, all_letters.find(line[0])] = 1
    else:
        tensor = torch.zeros(len(line)-1, n_letters)
        for idx, letter in enumerate(line[:-1]):
            tensor[idx, all_letters.find(letter)] = 1
    return Variable(tensor.cuda())
        
def targetTensor(line):
    letter_indexes = [all_letters.find(letter) for letter in line[1:]]
    return Variable(torch.LongTensor(letter_indexes).cuda())

def save_model(model, path):
    torch.save(model.state_dict(), path)
    
def load_model(model, path):
    model.load_state_dict(torch.load(path))

In [5]:
model = RNN(n_letters, hidden_dim, num_layers=num_layers, output_size=n_letters, drop_ratio=drop_ratio)
model = model.cuda()
criterion = nn.CrossEntropyLoss()

In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [7]:
n_epochs = 15
all_losses = []

print_every = 2000
init_hidden_every = 35

start_time = time.time()

loss_history = np.zeros((n_epochs, 2))

for epoch in range(n_epochs):
    print('Epoch # {}'.format(epoch))
    train_cnt = 0
    total_loss = 0.
    running_loss = 0. #for checking purpose
    
    model.hidden = model.init_hidden()
    
    #dataset.shuffle_data()
    #train_data, val_data = dataset.dataset()
#     model.train(True)
    for sentence in train_data:
        model.zero_grad()
        
        loss = 0.
        inputs = inputTensor(sentence)
        targets = targetTensor(sentence)
        
        preds = model(inputs)
        loss = criterion(preds, targets)
            
        loss.backward(retain_graph=True)
        optimizer.step()
        total_loss += loss
        train_cnt += 1
        
        if train_cnt % init_hidden_every == 0:
            model.hidden = model.init_hidden()
        
        if train_cnt % print_every == 0:
            total_loss /= print_every
            all_losses.append(total_loss)
            uptime = time.time() - start_time
            print('%dm:%ds -- [%d, %d], loss: %f' % (uptime/60, uptime%60, epoch, train_cnt, total_loss.data[0]))
            total_loss = 0
            
        running_loss += loss
    dataset_size = train_data.__len__()
    epoch_loss = running_loss / float(dataset_size)
    print('#, loss: {}'.format(epoch_loss))
            
#     model.train(False)
    for idx, dataloader in enumerate([train_data, val_data]):
        # compute loss
        running_loss = 0.
        for sentence in dataloader:
            loss = 0.
            inputs = inputTensor(sentence)
            targets = targetTensor(sentence)
            preds = model(inputs)
            loss = criterion(preds, targets)
            running_loss += loss.data[0]
            #
        # save loss
        dataset_size = dataloader.__len__()
        epoch_loss = running_loss / float(dataset_size)
        print('#{}, loss: {}'.format(idx, epoch_loss))
        # print(type(epoch_loss), type(epoch_acc))

        loss_history[epoch, idx] = epoch_loss
            
            
    scheduler.step()

Epoch # 0
0m:52s -- [0, 2000], loss: 3.117863
1m:41s -- [0, 4000], loss: 2.570372
2m:31s -- [0, 6000], loss: 2.176722
3m:21s -- [0, 8000], loss: 2.093189
4m:10s -- [0, 10000], loss: 1.852199
5m:2s -- [0, 12000], loss: 1.820882
5m:53s -- [0, 14000], loss: 1.830103
#, loss: Variable containing:
 2.1428
[torch.cuda.FloatTensor of size 1 (GPU 0)]

#0, loss: 2.574592954637307
#1, loss: 3.2947297951178536
Epoch # 1
8m:3s -- [1, 2000], loss: 2.190599
8m:51s -- [1, 4000], loss: 2.077456
9m:42s -- [1, 6000], loss: 1.771308
10m:30s -- [1, 8000], loss: 1.783957
11m:18s -- [1, 10000], loss: 1.643071
12m:6s -- [1, 12000], loss: 1.651070
12m:54s -- [1, 14000], loss: 1.674830
#, loss: Variable containing:
 1.7915
[torch.cuda.FloatTensor of size 1 (GPU 0)]

#0, loss: 2.4431941533999777
#1, loss: 3.1965441211879453
Epoch # 2
15m:5s -- [2, 2000], loss: 2.002563
15m:59s -- [2, 4000], loss: 1.940248
16m:50s -- [2, 6000], loss: 1.655522
17m:44s -- [2, 8000], loss: 1.676762
18m:36s -- [2, 10000], loss: 1.56

RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1503968623488/work/torch/lib/THC/generic/THCStorage.cu:66

In [8]:
# print(loss_history[8])

temp = np.concatenate((loss_history, np.array([loss_history[8]])), axis=0)
print(temp)
# print(all_losses)

[[ 2.57459295  3.2947298 ]
 [ 2.44319415  3.19654412]
 [ 2.35717108  3.11391485]
 [ 2.32063497  3.05223056]
 [ 2.28625979  3.00839267]
 [ 2.26066626  2.94519242]
 [ 2.13507388  2.74545424]
 [ 2.1113132   2.69373942]
 [ 2.0875864   2.65522327]
 [ 2.05366377  2.62001233]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 2.0875864   2.65522327]]


In [ ]:
fname = 'partc_drop_{}_loss_history.npy'.format(drop_ratio)
np.save(fname, temp)

In [ ]:
temp = np.load(fname)
plt.plot(range(16), temp)
plt.show()

In [ ]:
def test(start_with, max_length):
    model.hidden = model.init_hidden()
    seq = str(start_with)
    idx = []
    
    if len(start_with) > 1:
        inputs = inputTensor(seq)
        output = model(inputs)
    
    inputs = inputTensor(seq[-1])
    
    for i in range(max_length):
        output = model(inputs)
        #pr = F.softmax(output.data, 1)  //pytorch 3.0+
        pr = F.softmax(output.data)
        pr = pr.cpu().data.numpy().squeeze()
        pred = np.random.choice(n_letters, 1, p=pr)
        
        seq += all_letters[pred[0]]
        idx.append(pred[0])
        inputs = inputTensor(seq[-1])
    return seq, idx

def teacher_forcing_test(sentence):
    inputs = inputTensor(sentence)
    output = model(inputs)
    _, preds = torch.max(output.data, 1)
    seq = [all_letters[pred] for pred in preds ]
    return ''.join(seq)

In [ ]:
test_seq = val_data[0]
start_char = test_seq[:10]

seq, idx = test(start_char, 900)
print('Generated sequence: %s' % (seq))
#print('Sequence index: ', idx)
#print('Test sequence: %s' % test_seq)

In [ ]:
test_seq = val_data[99]
start_char = test_seq[:10]

seq, idx = test(start_char, 900)
print('Generated sequence: %s' % (seq))

In [ ]:
test_seq = val_data[0]
seq = teacher_forcing_test(test_seq)
print('Generated sequence: %s' % (seq))
print('Test sequence: %s' % test_seq)

In [ ]:
#save_model(model, 'full-model.pth')

In [ ]:
plt.figure()
plt.plot(all_losses)